<a href="https://colab.research.google.com/github/Itsuki-Hamano123/Quantum_prac/blob/master/a%2Bb%2Bc_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/量子コンピュータ練習/ナレッジ用に解く

/content/drive/My Drive/量子コンピュータ練習/ナレッジ用に解く


練習も兼ねて量子アニーリングを古典コンピュータ上でシュミレートしてみました。

シュミレートに使ったライブラリはPythonのwildqatです。

#量子アニーリングでa+b+c=3となるa,b,cを求めてみた（制約条件も加えてみた）

***
**今回の練習問題**

$a+b+c=3$を満たす整数($a,b,c$)の組み合わせを求める。（1-1～1-3）

$ab=2$の条件下で$a+b+c=3$を満たす整数($a,b,c$)の組み合わせを求める。（2-1～2-3）


**量子アニーリングで最適化問題を解く手順**

1. 問題をコスト関数（場合によっては制約条件も）で定式化
2. 定式化したものをQUBOに変換
3. QUBOから量子アニーリング（焼きなまし法）で基底状態（最適解）を求める
***

## 1-1.問題をコスト関数で定式化
最小二乗法を使って、問題を定式化します。

$0=\{(a+b+c)-3\}^2$

量子ビットはバイナリ値（0,1）を取るため、１量子ビットだけでは、整数値（$a,b,c$）を表現できません。そこで、複数の量子ビットを使った二進数表現で、$a,b,c$が整数値で表現できるよう定義します。（今回は簡略化のため必要最小限の量子ビットを用意）

$a=q_0+2q_1$、$b=q_2+2q_3$、$c=q_4+2q_5$

**先ほどのコスト関数に代入**

$0=[\{(q_0+2q_1)+(q_2+2q_3)+(q_4+2q_5)\}-3]^2$ 


## 1-2.定式化したものをQUBOに変換
コスト関数を展開(scipyを使ってみる)

In [0]:
from sympy import *
q0,q1,q2,q3,q4,q5=symbols("q0 q1 q2 q3 q4 q5")
f=(((q0+2*q1)+(q2+2*q3)+(q4+2*q5))-3)**2
print(expand(f))#式の展開

q0**2 + 4*q0*q1 + 2*q0*q2 + 4*q0*q3 + 2*q0*q4 + 4*q0*q5 - 6*q0 + 4*q1**2 + 4*q1*q2 + 8*q1*q3 + 4*q1*q4 + 8*q1*q5 - 12*q1 + q2**2 + 4*q2*q3 + 2*q2*q4 + 4*q2*q5 - 6*q2 + 4*q3**2 + 4*q3*q4 + 8*q3*q5 - 12*q3 + q4**2 + 4*q4*q5 - 6*q4 + 4*q5**2 - 12*q5 + 9


$q_0^2+4q_0q_1+2q_0q_2+4q_0q_3+2q_0q_4+4q_0q_5-6q_0+4q_1^2+4q_1q_2+8q_1q_3+4q_1q_4+8q_1q_5-12q_1+q_2^2+4q_2q_3+2q_2q_4+4q_2q_5-6q_2+4q_3^2+4q_3q_4+8q_3q_5-12q_3+q_4^2+4q_4q_5-6q_4+4q_5^2-12q_5+9$

**指数を外して式を整理**<br>
各量子ビットはバイナリ値を取り（$q_i\in(0,1)$）、二乗しても値が変わらない（$q_i=q_i^2$）ため、指数部が取れます。それにより式が少し整理できます。<br>
$-5q_0+4q_0q_1+2q_0q_2+4q_0q_3+2q_0q_4+4q_0q_5-8q_1+4q_1q_2+8q_1q_3+4q_1q_4+8q_1q_5-5q_2+4q_2q_3+2q_2q_4+4q_2q_5-8q_3+4q_3q_4+8q_3q_5-5q_4+4q_4q_5-8q_5+9$ 


**コスト関数をQUBOに変換**

例えば、$4q_0q_1$の場合、QUBOマトリクスの(0,1)要素に係数4を入れます。この要領で、QUBOマトリクスの各要素に値を入れていきます。

**できたQUBOマトリクス**

||$x_0$|$x_1$|$x_2$|$x_3$|$x_4$|$x_5$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$x_0$|-5|4|2|4|2|4|
|$x_1$|0|-8|4|8|4|8|
|$x_2$|0|0|-5|4|2|4|
|$x_3$|0|0|0|-8|4|8|
|$x_4$|0|0|0|0|-5|4|
|$x_5$|0|0|0|0|0|-8|

## 1-3.QUBOから量子アニーリング（焼きなまし法）で基底状態（最適解）を求める

In [0]:
#wildqatを持ってない人はインストール
!pip install wildqat

1-2で完成したQUBOの代入

In [0]:
import wildqat as wq
import numpy as np
a = wq.opt()

a.qubo = [[-5,4,2,4,2,4],[0,-8,4,8,4,8],[0,0,-5,4,2,4],[0,0,0,-8,4,8],[0,0,0,0,-5,4],[0,0,0,0,0,-8]]

#a.sa()

**量子アニーリングシュミレート（SA:焼きなまし法）で最適解を求めるのを100回繰り返す**

In [0]:
result=a.run(shots=100,sampler="fast",verbose=True)
wq.counter(result)

0 : [0, 1, 1, 0, 0, 0]
1 : [1, 0, 0, 1, 0, 0]
2 : [0, 1, 0, 0, 1, 0]
3 : [0, 0, 0, 0, 1, 1]
4 : [0, 0, 0, 0, 1, 1]
5 : [0, 0, 1, 0, 0, 1]
6 : [0, 0, 1, 1, 0, 0]
7 : [0, 1, 1, 0, 0, 0]
8 : [0, 0, 0, 1, 1, 0]
9 : [1, 0, 0, 1, 0, 0]
10 : [0, 0, 0, 0, 1, 1]
11 : [0, 0, 1, 0, 0, 1]
12 : [1, 0, 0, 1, 0, 0]
13 : [0, 1, 1, 0, 0, 0]
14 : [0, 0, 1, 1, 0, 0]
15 : [1, 0, 0, 0, 0, 1]
16 : [1, 0, 0, 1, 0, 0]
17 : [1, 0, 1, 0, 1, 0]
18 : [0, 1, 0, 0, 1, 0]
19 : [1, 0, 1, 0, 1, 0]
20 : [0, 0, 1, 1, 0, 0]
21 : [0, 1, 0, 0, 1, 0]
22 : [0, 0, 1, 1, 0, 0]
23 : [0, 0, 0, 1, 1, 0]
24 : [1, 0, 0, 0, 0, 1]
25 : [0, 0, 0, 1, 1, 0]
26 : [0, 0, 1, 1, 0, 0]
27 : [1, 1, 0, 0, 0, 0]
28 : [0, 0, 1, 0, 0, 1]
29 : [1, 0, 0, 0, 0, 1]
30 : [0, 0, 1, 0, 0, 1]
31 : [0, 1, 0, 0, 1, 0]
32 : [0, 0, 1, 1, 0, 0]
33 : [1, 0, 1, 0, 1, 0]
34 : [1, 0, 1, 0, 1, 0]
35 : [0, 0, 1, 0, 0, 1]
36 : [1, 0, 1, 0, 1, 0]
37 : [1, 1, 0, 0, 0, 0]
38 : [1, 1, 0, 0, 0, 0]
39 : [0, 1, 0, 0, 1, 0]
40 : [0, 0, 1, 0, 0, 1]
41 : [1, 1, 0, 0, 0, 0]
42

Counter({'000011': 9,
         '000110': 9,
         '001001': 16,
         '001100': 11,
         '010010': 9,
         '011000': 10,
         '100001': 7,
         '100100': 8,
         '101010': 10,
         '110000': 11})

上が一回ずつの実行結果で、下（Counter）が、実行結果とその出現回数を表しています。

1-1で定義した$a=q_0+2q_1$、$b=q_2+2q_3$、$c=q_4+2q_5$を元に結果を確認すると、
例えば'$001001$'の場合、$a=0,b=1,c=2$を表しています。他の結果も同様に確認すると、$a+b+c=3$を満たす整数($a,b,c$)が求められていることが分かります。

## 2-1.制約条件を定式化
次は、$ab=2$の条件下で$a+b+c=3$を満たす整数($a,b,c$)の組み合わせを求めてみます。

条件を制約条件（$ab=2$）として定式化してみます。そこで、$ab=2$の場合にエネルギーが最小になるように最小二乗法の式を作ります。

$0=\{(a*b)-2\}^2$

## 2-2.制約条件式をQUBOに変換
2-1で作成した制約条件式を展開。

In [0]:
from sympy import *
q0,q1,q2,q3=symbols("q0 q1 q2 q3")
f=((q0+2*q1)*(q2+2*q3)-2)**2
print(expand(f))#式の展開

q0**2*q2**2 + 4*q0**2*q2*q3 + 4*q0**2*q3**2 + 4*q0*q1*q2**2 + 16*q0*q1*q2*q3 + 16*q0*q1*q3**2 - 4*q0*q2 - 8*q0*q3 + 4*q1**2*q2**2 + 16*q1**2*q2*q3 + 16*q1**2*q3**2 - 8*q1*q2 - 16*q1*q3 + 4


$q_0^2q_2^2+4q_0^2q_2q_3+4q_0^2q_3^2+4q_0q_1q_2^2+16q_0q_1q_2q_3+16q_0q_1q_3^2-4q_0q_2-8q_0q_3+4q_1^2q_2^2+16q_1^2q_2q_3+16q_1^2q_3^2-8q_1q_2-16q_1q_3+4$<br>
各量子ビットがバイナリ値を取ることを考慮すると<br>
$q_0q_2+4q_0q_2q_3+4q_0q_3+4q_0q_1q_2+16q_0q_1q_2q_3+16q_0q_1q_3-4q_0q_2-8q_0q_3+4q_1q_2+16q_1q_2q_3+16q_1q_3-8q_1q_2-16q_1q_3+4$<br>
各量子ビットがバイナリ値を取ることを考慮し式を整理<br>
$-3q_0q_2+4q_0q_2q_3-4q_0q_3+4q_0q_1q_2+16q_0q_1q_2q_3+16q_0q_1q_3-4q_1q_2+16q_1q_2q_3+4$

# ★人類にはほぼ解けない多体問題
$-3q_0q_2$の場合、QUBOマトリクスの要素（0,2）に$-3$を入れていました。

しかし、$4q_0q_2q_3$や$16q_0q_1q_2q_3$などは、3つの要素が掛け合わされており、QUBOに落とし込むことが出来ません。

３体の要素が相互作用する問題を3体問題と呼び、さらに３体問題以上の問題は、総じて多体問題と呼ばれます。ポアンカレによると、この多体問題は、特別な例を除いて解を求めることが出来きないみたいです。（多体問題が解ければ、３つ以上の惑星間の計算が正確に出来るそうです。）

そこで、多体問題に出会ってしまった時には、補助項を設けるなどして、いくつかの要素に制限を設け2体問題に落とし込んだり、２体問題にならないよう問題設定を見直す必要があります。

## ★2-1.２体問題になるよう制約条件式作り直し
多体問題にならないよう、問題の条件を少し変更します。$ab=2$の代わりに、$a\in(1,3),b\in(2,3)$

（$q_0=1,q_3=1$）を条件として以下の制約条件式を作ります。

$(q_0-1)^2+(q_3-1)^2$

この制約条件式は、$q_0=1$かつ$q_3=1$となったときにエネルギーが最小になります。

## ★2-2.作り直した制約条件式をQUBOに変換
||$x_0$|$x_1$|$x_2$|$x_3$|$x_4$|$x_5$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$x_0$|-1|0|0|0|0|0|
|$x_1$|0|0|0|0|0|0|
|$x_2$|0|0|0|0|0|0|
|$x_3$|0|0|0|-1|0|0|
|$x_4$|0|0|0|0|0|0|
|$x_5$|0|0|0|0|0|0||

## 2-3. コスト関数のQUBOと制約条件のQUBOを調合し、量子アニーリング（焼きなまし法）で基底状態（最適解）を求める
コスト関数のQUBOと制約条件のQUBOを調合します。それぞれをどのくらい割合で組み合わせるかはハイパーパラメータで設定します。

**コスト関数のQUBOを0.5倍、制約条件のQUBOを0.5倍にして調合**

In [0]:
import wildqat as wq
import numpy as np
a = wq.opt()

cost_qubo = np.array([[-5,4,2,4,2,4],[0,-8,4,8,4,8],[0,0,-5,4,2,4],[0,0,0,-8,4,8],[0,0,0,0,-5,4],[0,0,0,0,0,-8]])
r_qubo=np.array([[-1,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,-1,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]])
#a.sa()

In [0]:
qubo=(0.5*cost_qubo)+(0.5*r_qubo)
print(qubo)

[[-3.   2.   1.   2.   1.   2. ]
 [ 0.  -4.   2.   4.   2.   4. ]
 [ 0.   0.  -2.5  2.   1.   2. ]
 [ 0.   0.   0.  -4.5  2.   4. ]
 [ 0.   0.   0.   0.  -2.5  2. ]
 [ 0.   0.   0.   0.   0.  -4. ]]


**量子アニーリングシュミレート（焼きなまし法）で解く**

In [0]:
a.qubo=qubo
result=a.run(shots=100,sampler="fast",verbose=True)
wq.counter(result)

0 : [1, 0, 0, 1, 0, 0]
1 : [1, 1, 0, 0, 0, 0]
2 : [1, 0, 0, 1, 0, 0]
3 : [1, 0, 0, 1, 0, 0]
4 : [1, 0, 0, 1, 0, 0]
5 : [1, 0, 0, 1, 0, 0]
6 : [1, 0, 0, 1, 0, 0]
7 : [1, 0, 0, 1, 0, 0]
8 : [1, 0, 0, 1, 0, 0]
9 : [1, 0, 0, 1, 0, 0]
10 : [1, 0, 0, 1, 0, 0]
11 : [1, 0, 0, 1, 0, 0]
12 : [1, 0, 0, 1, 0, 0]
13 : [1, 0, 0, 1, 0, 0]
14 : [1, 0, 0, 1, 0, 0]
15 : [1, 0, 0, 1, 0, 0]
16 : [1, 0, 0, 1, 0, 0]
17 : [1, 0, 0, 1, 0, 0]
18 : [1, 0, 0, 1, 0, 0]
19 : [1, 0, 0, 1, 0, 0]
20 : [1, 0, 0, 1, 0, 0]
21 : [1, 0, 0, 1, 0, 0]
22 : [1, 0, 0, 1, 0, 0]
23 : [1, 0, 0, 1, 0, 0]
24 : [1, 0, 0, 1, 0, 0]
25 : [1, 0, 0, 1, 0, 0]
26 : [1, 0, 0, 1, 0, 0]
27 : [1, 0, 0, 1, 0, 0]
28 : [1, 0, 0, 0, 0, 1]
29 : [1, 0, 0, 1, 0, 0]
30 : [1, 0, 0, 1, 0, 0]
31 : [1, 0, 1, 0, 1, 0]
32 : [1, 0, 1, 0, 1, 0]
33 : [1, 0, 0, 1, 0, 0]
34 : [1, 0, 0, 1, 0, 0]
35 : [1, 0, 0, 1, 0, 0]
36 : [1, 0, 0, 1, 0, 0]
37 : [1, 0, 0, 1, 0, 0]
38 : [1, 0, 0, 1, 0, 0]
39 : [1, 0, 0, 1, 0, 0]
40 : [1, 0, 0, 1, 0, 0]
41 : [1, 0, 0, 1, 0, 0]
42

Counter({'100001': 5, '100100': 85, '101010': 5, '110000': 5})

100回シュミレートした結果の内、85回と一番多く出現した"100100"は、$a=1,b=2,c=0$を表現しており、

$ab=2$の条件下で$a+b+c=3$を満たす整数($a,b,c$)の組み合わせを求めることが出来ました。

**コスト関数のQUBOを0.3倍、制約条件のQUBOを0.7倍にして調合した場合**

In [0]:
qubo=(0.3*cost_qubo)+(0.7*r_qubo)
print(qubo)

[[-2.2  1.2  0.6  1.2  0.6  1.2]
 [ 0.  -2.4  1.2  2.4  1.2  2.4]
 [ 0.   0.  -1.5  1.2  0.6  1.2]
 [ 0.   0.   0.  -3.1  1.2  2.4]
 [ 0.   0.   0.   0.  -1.5  1.2]
 [ 0.   0.   0.   0.   0.  -2.4]]


In [0]:
a.qubo=qubo
result=a.run(shots=100,sampler="fast",verbose=True)
wq.counter(result)

0 : [1, 0, 0, 1, 0, 0]
1 : [1, 0, 0, 1, 0, 0]
2 : [1, 0, 0, 1, 0, 0]
3 : [1, 0, 0, 1, 0, 0]
4 : [1, 0, 0, 1, 0, 0]
5 : [1, 0, 0, 1, 0, 0]
6 : [1, 1, 0, 0, 0, 0]
7 : [1, 0, 0, 1, 0, 0]
8 : [1, 0, 0, 1, 0, 0]
9 : [1, 0, 0, 1, 0, 0]
10 : [1, 0, 0, 1, 0, 0]
11 : [1, 0, 0, 1, 0, 0]
12 : [1, 0, 0, 1, 0, 0]
13 : [1, 0, 0, 1, 0, 0]
14 : [1, 0, 0, 1, 0, 0]
15 : [1, 0, 0, 1, 0, 0]
16 : [1, 0, 0, 1, 0, 0]
17 : [1, 0, 0, 1, 0, 0]
18 : [1, 0, 0, 1, 0, 0]
19 : [1, 0, 0, 1, 0, 0]
20 : [1, 0, 0, 1, 0, 0]
21 : [1, 0, 0, 1, 0, 0]
22 : [1, 0, 0, 1, 0, 0]
23 : [1, 0, 0, 1, 0, 0]
24 : [1, 0, 0, 1, 0, 0]
25 : [1, 0, 0, 1, 0, 0]
26 : [1, 0, 0, 1, 0, 0]
27 : [1, 0, 0, 1, 0, 0]
28 : [1, 0, 0, 1, 0, 0]
29 : [1, 0, 0, 1, 0, 0]
30 : [1, 0, 0, 1, 0, 0]
31 : [1, 0, 0, 1, 0, 0]
32 : [1, 0, 0, 1, 0, 0]
33 : [1, 0, 0, 1, 0, 0]
34 : [1, 0, 0, 1, 0, 0]
35 : [1, 0, 0, 1, 0, 0]
36 : [1, 0, 0, 1, 0, 0]
37 : [1, 0, 0, 1, 0, 0]
38 : [1, 0, 0, 1, 0, 0]
39 : [1, 0, 0, 1, 0, 0]
40 : [1, 0, 0, 1, 0, 0]
41 : [1, 0, 0, 1, 0, 0]
42

Counter({'100001': 1, '100100': 98, '110000': 1})

精度上がりました。